# Omniscient Policy / Oracle

This notebook represents an omniscient policy that knows all of the probability distributions. This algorithm knows, every step of the way, the best decision based on its knowledge of the true distributions. It does not have to learn anything. The oracle has optimal parameters $\theta$, hence it is expected to maximize reward in fewer rounds. 

In [10]:
# Configure / validate these before you run: File Path, alpha, confidence_threshold ,to_csv. 

In [11]:
import numpy as np
import pandas as pd

# Contexts

In [12]:
import os,pickle

# file_path = os.path.join(os.path.curdir, '..' , 'dataset' , 'very_small')
file_path = os.path.join(os.path.curdir, '..' , 'dataset' , 'small')
# file_path = os.path.join(os.path.curdir, '..' , 'dataset' , 'medium')
# file_path = os.path.join(os.path.curdir, '..' , 'dataset' , 'large')
# file_path = os.path.join(os.path.curdir, '..' , 'dataset' , 'very_large')
'''
Context data for learning
'''
class Context:
    """
    Contextual information required by contextual bandit algorithms to make better predictions. It enscapsulates all data
    about the student , topics & content to experiment with the oracle
    """
   
    def getStudentContext(self):
        """
        Student Preferences: 
        Visual (S_V) , Text (S_T) , Demo-based (S_D) , Practical (S_P), Step-by-step (S_S) ,Activity / Task based (S_AT), 
        Lecture (S_L) , Audio (S_A) , Self-evaluation (S_SE) , Pre-assessment (S_PA)
        Students preference to learning via various ways are evaluated on a scale from 0 to 1, rather being binary. 
        """
        return self.studentContext
    
    def setStudentContext(self):
        """
        Load the student data
        """
        with open(os.path.join(file_path , 'student.pickle'), 'rb') as student_file:
            self.studentContext= pickle.load(student_file)
    
    def getContentContext(self):
        """
        Content Features 
        Ease of understanding (C_E) , Simple / Intuitive (C_I) , Surface / In-depth (C_ID) , Brief / Concise (C_C), 
        Thorough (C_T), Preference / Well reviewed / Well rated (C_R) , Theoritical / Abstract (C_A), 
        Practical / Hands on (C_P), Experimental / Task-based (C_ETB)
        Content preference to learning via various ways are evaluated on a scale from 0 to 1, rather being binary. 
        """
        return self.contentContext
   
    def setContentContext(self):
        """
        Load the content data
        """
        with open(os.path.join(file_path ,'content.pickle'), 'rb') as content_file:
            self.contentContext= pickle.load(content_file)
        
    def getTopic(self):
        """
        Gives the topics part of the course.
        """
        return self.topic
    
    def setTopic(self):
        """
        Loads the topics part of the course
        """
        with open(os.path.join(file_path ,'topic.pickle'), 'rb') as topic_file:
            self.topic = pickle.load(topic_file)
    
    def getTopicContent(self):
        """
         Gets the topic content. topic_content is a map of topics to content. So for every topic, it gives the content 
         available for the topic. In education parlance, for any given topic, it shows the different ways of teaching this
         topic (via contents)
        """
        return self.topic_content
    
    def setTopicContent(self):
        """
        Sets the topic_content variable to the one in the serialized object. topic_content is a map of topics to content. So
        for every topic, it gives the content available for the topic. In education parlance, for any given topic, it shows
        the different ways of teaching this topic (via contents)
        """
        with open(os.path.join(file_path ,'topic_content.pickle'), 'rb') as topic_content_file:
            self.topic_content= pickle.load(topic_content_file)
                
    def prepareContext(self,studentContext,contentContext):
        """
           Given the student & content context available for a round, this method combines them to form a single contextual
           variable
           
           Inputs : 
           
           studentContext: Student contextual information.
           contentContext: Contents contextual information. 
           
           Returns :
           
           context : A combined output of student & content context.
        """
        context = pd.DataFrame() 
        for content in list(contentContext.index):
            c = pd.Series()
            c = c.append([studentContext,contentContext.loc[content]]) # Combine student & content. 
            c['Content_id'] = content
            context = context.append(c, ignore_index=True)
        context = context.set_index('Content_id')
        return context
    
    def loadData(self):
        """
        Method used to test data retrieval. Data generator handles the data generation. This method checks we can retrieve
        data. This is a dummy method used to test data retrieval. Its not invoked in the main program.
        """
        self.setStudentContext()
        self.setContentContext()
        self.setTopic()
        self.setTopicContent()
        print("Student Context Shape : ", self.getStudentContext().shape)
        print("Content Context Shape : ", self.getContentContext().shape)
        print("**********************************")
        print(self.getStudentContext())
        print(type(self.getStudentContext()))
        print('*********************************')
        print(self.getContentContext())
        print(type(self.getContentContext()))
        print('*********************************')
        print(self.getTopic())
        print(type(self.getTopic()))
        print('*********************************')
        print(self.getTopicContent())
        print(type(self.getTopicContent()))
                
c_test = Context()
c_test.loadData()

Student Context Shape :  (100, 10)
Content Context Shape :  (329, 9)
**********************************
     S_V   S_T   S_D   S_P   S_S  S_AT   S_L   S_A  S_SE  S_PA
0   0.87  0.82  0.88  0.36  0.60  0.06  0.66  0.56  0.66  0.07
1   0.60  0.00  0.97  0.26  0.68  0.19  0.77  0.09  0.57  0.41
2   0.53  0.15  0.24  0.23  0.91  0.68  0.22  0.50  0.92  0.85
3   0.11  0.28  0.61  0.77  0.21  0.53  0.75  0.87  0.69  0.08
4   0.15  0.32  0.67  0.33  0.31  0.58  0.27  0.80  0.79  0.97
5   0.67  0.77  0.02  0.86  0.99  0.52  0.39  0.34  0.57  0.99
6   0.09  0.02  0.66  0.12  0.28  0.07  0.33  0.76  0.99  0.63
7   0.07  0.16  0.07  0.24  0.00  0.67  0.04  0.95  0.28  0.37
8   0.21  0.21  0.32  0.08  0.60  0.89  0.26  0.20  0.04  0.04
9   0.38  0.03  0.71  0.88  0.15  0.51  0.79  0.43  0.02  0.65
10  0.52  0.61  0.71  0.13  0.68  0.31  0.32  0.69  0.70  0.74
11  0.37  0.60  0.58  0.94  0.19  0.69  0.40  0.83  0.47  0.57
12  0.12  0.48  0.59  0.23  0.89  0.36  0.49  0.41  0.93  1.00
13  0.30  0.53

# Skip Classifier

In [13]:
# Online Stochastic Gradient Descent. This classifier decides whether or not to skip to the next topic. 
# TO-DO : Change loss functions (Log,Hinge,Others) to find if they impact performance. Try different values of parameters 
# For instance SGD has a parameter alpha, SVM has a parameter C. To optimize, you can train a mini-batch of samples, 
# rather than one data point at a time. Try different values of learning_rate . Look at the class_weight parameter if you 
# want to give more weight to samples of one class over the other. Need to understand about warm_start parameter
# We need to record predictions made by the classifier to evaluate its performance over rounds.
from sklearn import linear_model
class SkipClassifier:
    """
    A classifier which gives prediction, whether or not to move to the next topic. This is important, because we want 
    students to learn content which the algorithm is confident would help the student learn. The skip classifier is trained
    online, hence we use a confidence threshold, to be conservative & minimize skipping topics. Skipping is not preferred, 
    but if the classifier is confident the next round would help gain higher rewards, then we should skip. Ideally, we want 
    to consider skipping after the first pulled arm has failed, to avoid frustrating the student. 
    """
    def __init__(self):
        self.clf = linear_model.SGDClassifier()
        self.clf.partial_fit(np.array([[0,0,0,0,0,0,0,0,0,0,0,0]]),np.array([0]),classes=np.array([0,1])) # Used to initialize the skip classifier
        
    def check_fitted(self,clf): 
        """
        Check if the classifier is fit before asking for prediction. Our classifier is trained in online mode, hence it would
        be asked to predict before fitting. This method makes sure we only ask for prediction after a data point has been 
        fit to the estimator/model
        """
        return hasattr(clf, "classes_")
    
    def train(self,student,pta,next_topic_pta,label):
        """
        Used to train the classifier in online mode, over every data point. In future we might want to consider training in 
        mini-batches, rather than for every data point. 
        """
        X = pd.Series()
        X = X.append([student,pd.Series([pta,next_topic_pta],index=['pta','next_topic_pta'])])
        X = np.array([X.values])
        Y = np.array([label])
        self.clf = self.clf.partial_fit(X,Y)
               
    def predict(self,student,pta,next_topic_pta):
        """
        Gets predictions from the classifier, along with the confidence score to help determine the reliability / confidence
        level of the prediction being made. 
        """
        X = pd.Series()
        X = X.append([student,pd.Series([pta,next_topic_pta],index=['pta','next_topic_pta'])])
        if self.check_fitted(self.clf):
            Y = self.clf.predict([X.values])[0]
            confidence_score = self.clf.decision_function([X.values])[0]
        else:
            Y = 0
            confidence_score = 0
        return int(Y) , confidence_score 

# SkipTopic

In [14]:
class SkipTopic:
    """
    A wrapper around the Skip Classifier to validate the inputs, before sending it to Skip Classifier for prediction. 
    It post-processes the results of the prediction made by skip classifier to check for confidence threshold, 
    before sending out the decision to skip or not. 
    """
    def __init__(self):
        """
        Initializes the SkipTopic class & sets confidence threshold to make confident skip decisions.
        """
        self.skipClassifier = SkipClassifier()
        self.confidence_threshold = 10 # It the confidence score returned by the classifier is greater than this, then we trust in the decision made by the classifier. 
                
    def skipTopic(self,student,pta,topic_number,context_obj,topic_content,oracle):
        """
        Pre-validates the topic number before asking the skip classifier for a prediction. Then checks the confidence 
        of the prediction before sending out the decision to skip or not. 
        """
        contentContext = context_obj.getContentContext() # Get the content dataframe.
        topic = context_obj.getTopic() # Get the topic list. 
        current_topic_index = topic.index(topic_number) # Get the index number of the current topic
        next_topic_index = current_topic_index + 1
        next_topic = '' # Initialized to make it accessible outside the if statement. 
        if next_topic_index < len(topic): # Check to see if we're going out of bounds
            next_topic = topic[next_topic_index]
            next_topic_contents = topic_content[next_topic]
            t_c = contentContext.loc[next_topic_contents]
            X = context_obj.prepareContext(student,t_c)
            arm_pulled , next_topic_pta = oracle.expectedPayoff(X,next_topic_contents)
        else:
            # Will be going out of bounds. Current topic is the last topic. No more topics to complete. 
            next_topic_pta = 0
        actual_decision , confidence_score = self.skipClassifier.predict(student,pta,next_topic_pta)
        if actual_decision and confidence_score > self.confidence_threshold:
            skip_decision = 1
        else:
            skip_decision = 0
        return actual_decision,confidence_score,skip_decision,next_topic_pta

    def setLabel(self,actual_payoff):
        """
        Sets the label for training the skip classifier
        """
        if actual_payoff == 0:
            label = 1
        if actual_payoff == 1:
            label = 0
        return label
    
    def train(self,student,pta,pta_next_topic,label):
        """
        Training the skip classifier
        """
        self.skipClassifier.train(student,pta,pta_next_topic,label)              

# Omniscient Policy / Oracle

In [15]:
class Oracle :
    """
    It has the optimal parameters to maximize rewards. The learning algorithm updates its parameters to emulate its parameters
    It is an omniscient policy that knows all of the probability distributions. This is the algorithm which, every step of 
    the way, makes the best decision based on its knowledge of the true distributions (it does not have to learn anything). 
    """
    def __init__(self):
        """
        Initalizes parameters for the omniscient policy. 
        """
        self.rounds = 0 # Number of round played
        self.rounds_data = pd.DataFrame() # Rounds data required for Skip Algorithm
        self.skipTopic = SkipTopic()
    
    def setParameters(self, features , arms): # Setting optimal parameter theta
        """
        Sets the optimal parameters for the omniscient policy. 
        """
        parameters = np.random.uniform(size=(len(arms) , len(features)))
        # Normalize parameters
        for i in range(parameters.shape[0]): # Have it in a list comprehension.
            parameters[i] = parameters[i] / np.sum(parameters[i])
        self.theta_df = pd.DataFrame(data = parameters ,  index = arms , columns = features , dtype= np.float)
    
    def expectedPayoff(self,contexts,arms):
        """
        Gives the max expected pay-off for a round with the given context & available arms. The arm is not pulled up here as we 
        also depend of the decision from the skip classifer before the arm is actually pulled.         
        
        Input : 
        
        contexts : Contextual data available in the round. Its a combination of student & content context
        arms : Arms / Content available in this round. 
        
        Returns : 
        
        arm_pulled : The arm that should be pulled 
        expected_payoff : Expected pay-off for the pulled suggested to be pulled. 
        
        """
        arms_payoff = list()
        for arm in arms:
            arm_theta = self.theta_df.loc[arm]
            X = contexts.loc[arm]
            pta = pd.Series.dot(X,arm_theta) # Vector dim : (1 * d) (d * 1).
            arms_payoff.append(pta)
        arm_index = np.argmax(arms_payoff)
        arm_pulled = arms[arm_index]
        expected_payoff = np.max(arms_payoff)
        return arm_pulled,np.round(expected_payoff,2)

# Simulator

In [16]:
import threading
from scipy.stats import bernoulli

class Simulator:
    
    def __init__(self):
        self.context = Context()
        self.context.setStudentContext()
        self.context.setContentContext()
        self.context.setTopic()
        self.context.setTopicContent()
        self.oracle = Oracle()
        self.skipTopic = SkipTopic()
        self.simulator_lock = threading.Lock()
        self.rounds=0
        self.logs = pd.DataFrame(columns = ['student_number','topic','arm_pulled','pay-off','pay-off_next_topic','actual_decision','skip_decision'
                                            ,'reward']) 

    def getPayoff(self,pta):
        """
        Student shares feedback about the content / understanding of the topic. 
        
        Input : 
        
        pta : Payoff at round 't' for pulling an arm. 
        
        Returns : 
        
        reward : Reward / Feedback from student for the content shown / arm pulled
        """
        reward = bernoulli.rvs(size=1,p=pta)[0] # Simulate student's response
        return reward
    
    def takeCourse(self,student_number,studentContext,contentContext,topic,topic_content):
        """
        This method simulates students taking a course. As part of it, students are presented content for various topics. 
        Students share their feedback, based on which we either move to the next topic or remain on the same topic.  
        We get the expected pay-off from the oracle. We then decide whether to skip or remain on the same topic.
        If skip is true, then the student moves to the next topic, else the student remains on the same topic, shares feedback on 
        the content & we train the skip classifier with this feedback. This method drives the flow of the system, hence key 
        data elements available in this method are logged for analysis.
        
        Inputs : 
        
        student_number : Student Id 
        studentContext : Student context vector. 
        contentContext : Contents context. This has context of all contents for the topic. 
        topic : All the topics to be taught as part of the course. 
        topic_content : Relates all topics to the contents available for every topic     
         
        """
        for i in topic:
            skip_enabled = False # Done to disable skipping without attempting to teach a student. 
            contents = topic_content[i] # You now have all arm associated with the topic 't'
            t_c = contentContext.loc[contents]
            contexts = self.context.prepareContext(studentContext,t_c)
            arms = list(t_c.index)
            while arms:
                arm , pta = self.oracle.expectedPayoff(contexts,arms)
                actual_decision , confidence_score, skip_decision , pta_next_topic = self.skipTopic.skipTopic(studentContext,pta,i,self.context,topic_content,self.oracle)
                if skip_decision and skip_enabled:
                    log = pd.Series([student_number,i,arm,pta,pta_next_topic,actual_decision,confidence_score,skip_decision], 
                                        index=['student_number','topic','arm_pulled','pay-off',
                                                'pay-off_next_topic','actual_decision','confidence_score','skip_decision']) # Print log for this round
                    with self.simulator_lock:
                        print('We\'re skipping. Student {0} is on topic {1} was expected to be shown content {2}. Expected Pay-off of this arm is {3}, compared to expected pay-off of next round is {4}. Actual decision was {5} with confidence {6} Decision of skip classifier is {7}'
                          .format(student_number,i,arm,pta,pta_next_topic,actual_decision,confidence_score,skip_decision))                    
                        self.logs = self.logs.append(log , ignore_index=True) # Log in a file
                    break # Decision is to skip. Hence, we won't pull the arm. 
                else:
                    actual_payoff = self.getPayoff(pta)
                    log = pd.Series([student_number,i,arm,pta,pta_next_topic,actual_decision,confidence_score,skip_decision,actual_payoff], 
                                        index=['student_number','topic','arm_pulled','pay-off',
                                                'pay-off_next_topic','actual_decision','confidence_score','skip_decision','reward']) # Print log for this round
                    with self.simulator_lock:
                        self.rounds+=1
                        print('Student {0} is on topic {1} is shown content {2} feedback recd is {3}. Expected Pay-off of this arm is {4}, compared to expected pay-off of next round is {5}. Actual decision was {6} with confidence {7}. Decision of skip classifier is {8}'
                              .format(student_number,i,arm,actual_payoff,pta,pta_next_topic,actual_decision,confidence_score,skip_decision))
                        self.logs = self.logs.append(log , ignore_index=True) # Log in a file
                    label = self.skipTopic.setLabel(actual_payoff) # Set Label
                    self.skipTopic.train(studentContext,pta,pta_next_topic,label)
                if actual_payoff != 1:
                    arms.remove(arm)
                    skip_enabled = True
                else:
                    break # Move to the next topic 

    def main(self):
        """
        Its the main method. Its in the name :)
        """
        studentContext = self.context.getStudentContext() # Student dataframe
        contentContext = self.context.getContentContext() # Content Dataframe
        topic = self.context.getTopic() # List of topics. 
        topic_content = self.context.getTopicContent() # Topics Data, which includes topics to content mapping.
        features = list(studentContext.columns) + list(contentContext.columns)
        self.oracle.setParameters(features , contentContext.index) 
        student_thread = list() # Keep track of students taking the course. 
        for student_number , student in studentContext.iterrows():
            t = threading.Thread(target=self.takeCourse, args=(student_number,student,contentContext,topic,topic_content))
            student_thread.append(t)
            # Some threads do background tasks, like sending keepalive packets, or performing periodic garbage collection, or 
            # whatever. These are only useful when the main program is running, and it's okay to kill them off once the other, 
            # non-daemon, threads have exited. Once the main thread finishes & one of the student is still working through the course. 
            # we will wait for the student to complete the course, since the main thread is completed. We want all students 
            # to complete the course. Hence, setting daemon to False
            t.daemon = False # classifying as a daemon, so they will die when the main dies
            t.start() # begins, must come after daemon definition
        for t in student_thread: # This is done to ensure, we proceed to save the logs only after all students have completed the course. 
            t.join()
        self.logs.to_csv('logs_oracle_small_CT10',index=False)
        print('Total Number of rounds : ', self.rounds)  
    
simulator = Simulator()
simulator.main()

Student 1 is on topic T_1 is shown content C_1_4 feedback recd is 1. Expected Pay-off of this arm is 0.59, compared to expected pay-off of next round is 0.57. Actual decision was 0 with confidence -10.0. Decision of skip classifier is 0
Student 2 is on topic T_1 is shown content C_1_4 feedback recd is 1. Expected Pay-off of this arm is 0.65, compared to expected pay-off of next round is 0.63. Actual decision was 0 with confidence -10.0. Decision of skip classifier is 0
Student 3 is on topic T_1 is shown content C_1_4 feedback recd is 1. Expected Pay-off of this arm is 0.59, compared to expected pay-off of next round is 0.62. Actual decision was 0 with confidence -10.0. Decision of skip classifier is 0
Student 60 is on topic T_1 is shown content C_1_4 feedback recd is 0. Expected Pay-off of this arm is 0.6, compared to expected pay-off of next round is 0.51. Actual decision was 0 with confidence -10.0. Decision of skip classifier is 0
Student 44 is on topic T_1 is shown content C_1_4 fe

Student 82 is on topic T_1 is shown content C_1_4 feedback recd is 0. Expected Pay-off of this arm is 0.55, compared to expected pay-off of next round is 0.51. Actual decision was 1 with confidence 14.859526556842484. Decision of skip classifier is 1
Student 12 is on topic T_1 is shown content C_1_1 feedback recd is 0. Expected Pay-off of this arm is 0.65, compared to expected pay-off of next round is 0.61. Actual decision was 0 with confidence -20.45137051638344. Decision of skip classifier is 0
Student 14 is on topic T_1 is shown content C_1_4 feedback recd is 0. Expected Pay-off of this arm is 0.65, compared to expected pay-off of next round is 0.58. Actual decision was 0 with confidence -34.112453287647654. Decision of skip classifier is 0
Student 59 is on topic T_1 is shown content C_1_4 feedback recd is 0. Expected Pay-off of this arm is 0.71, compared to expected pay-off of next round is 0.67. Actual decision was 1 with confidence 18.863479126012443. Decision of skip classifier 

Student 78 is on topic T_1 is shown content C_1_7 feedback recd is 0. Expected Pay-off of this arm is 0.66, compared to expected pay-off of next round is 0.63. Actual decision was 0 with confidence -42.320039658472055. Decision of skip classifier is 0
Student 31 is on topic T_1 is shown content C_1_4 feedback recd is 1. Expected Pay-off of this arm is 0.61, compared to expected pay-off of next round is 0.65. Actual decision was 1 with confidence 13.800193059936925. Decision of skip classifier is 1
Student 17 is on topic T_1 is shown content C_1_4 feedback recd is 0. Expected Pay-off of this arm is 0.7, compared to expected pay-off of next round is 0.67. Actual decision was 1 with confidence 18.490692570220958. Decision of skip classifier is 1
Student 42 is on topic T_1 is shown content C_1_4 feedback recd is 1. Expected Pay-off of this arm is 0.65, compared to expected pay-off of next round is 0.67. Actual decision was 1 with confidence 23.182171512434476. Decision of skip classifier i

Student 13 is on topic T_1 is shown content C_1_7 feedback recd is 1. Expected Pay-off of this arm is 0.55, compared to expected pay-off of next round is 0.58. Actual decision was 0 with confidence -53.84468465486312. Decision of skip classifier is 0
Student 9 is on topic T_1 is shown content C_1_14 feedback recd is 1. Expected Pay-off of this arm is 0.56, compared to expected pay-off of next round is 0.59. Actual decision was 0 with confidence -39.61859769834139. Decision of skip classifier is 0
Student 79 is on topic T_1 is shown content C_1_7 feedback recd is 1. Expected Pay-off of this arm is 0.58, compared to expected pay-off of next round is 0.6. Actual decision was 0 with confidence -53.21956388191626. Decision of skip classifier is 0
We're skipping. Student 57 is on topic T_1 was expected to be shown content C_1_7. Expected Pay-off of this arm is 0.54, compared to expected pay-off of next round is 0.55. Actual decision was 1 with confidence 26.794827639045206 Decision of skip c

Student 98 is on topic T_2 is shown content C_2_1 feedback recd is 1. Expected Pay-off of this arm is 0.58, compared to expected pay-off of next round is 0.56. Actual decision was 0 with confidence -58.18559491105938. Decision of skip classifier is 0
Student 47 is on topic T_2 is shown content C_2_1 feedback recd is 1. Expected Pay-off of this arm is 0.62, compared to expected pay-off of next round is 0.57. Actual decision was 0 with confidence -36.29278497434369. Decision of skip classifier is 0
Student 25 is on topic T_2 is shown content C_2_1 feedback recd is 0. Expected Pay-off of this arm is 0.65, compared to expected pay-off of next round is 0.6. Actual decision was 0 with confidence -43.64816120556284. Decision of skip classifier is 0
Student 69 is on topic T_1 is shown content C_1_7 feedback recd is 0. Expected Pay-off of this arm is 0.62, compared to expected pay-off of next round is 0.61. Actual decision was 0 with confidence -36.898392045821375. Decision of skip classifier i

...

Student 34 is on topic T_25 is shown content C_25_1 feedback recd is 1. Expected Pay-off of this arm is 0.6, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 3.3709460498760233. Decision of skip classifier is 0
Student 3 is on topic T_23 is shown content C_23_13 feedback recd is 0. Expected Pay-off of this arm is 0.52, compared to expected pay-off of next round is 0.56. Actual decision was 1 with confidence 8.87396306300584. Decision of skip classifier is 0
Student 13 is on topic T_25 is shown content C_25_14 feedback recd is 0. Expected Pay-off of this arm is 0.49, compared to expected pay-off of next round is 0. Actual decision was 0 with confidence -1.2384418012466378. Decision of skip classifier is 0
Student 63 is on topic T_25 is shown content C_25_1 feedback recd is 0. Expected Pay-off of this arm is 0.55, compared to expected pay-off of next round is 0. Actual decision was 0 with confidence -7.703251497185379. Decision of skip classifier is 

Student 51 is on topic T_21 is shown content C_21_3 feedback recd is 0. Expected Pay-off of this arm is 0.54, compared to expected pay-off of next round is 0.52. Actual decision was 0 with confidence -1.0761933398401116. Decision of skip classifier is 0
Student 40 is on topic T_20 is shown content C_20_9 feedback recd is 0. Expected Pay-off of this arm is 0.58, compared to expected pay-off of next round is 0.58. Actual decision was 0 with confidence -2.037788613104505. Decision of skip classifier is 0
Student 9 is on topic T_25 is shown content C_25_14 feedback recd is 0. Expected Pay-off of this arm is 0.52, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 9.575842954736775. Decision of skip classifier is 0
Student 79 is on topic T_24 is shown content C_24_2 feedback recd is 0. Expected Pay-off of this arm is 0.51, compared to expected pay-off of next round is 0.57. Actual decision was 1 with confidence 1.3443595194129117. Decision of skip classif

Student 33 is on topic T_25 is shown content C_25_5 feedback recd is 1. Expected Pay-off of this arm is 0.45, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 1.4019324310123178. Decision of skip classifier is 0
Student 82 is on topic T_24 is shown content C_24_6 feedback recd is 1. Expected Pay-off of this arm is 0.48, compared to expected pay-off of next round is 0.53. Actual decision was 1 with confidence 1.1978739875270774. Decision of skip classifier is 0
Student 44 is on topic T_25 is shown content C_25_14 feedback recd is 1. Expected Pay-off of this arm is 0.48, compared to expected pay-off of next round is 0. Actual decision was 0 with confidence -2.9539085983000284. Decision of skip classifier is 0
Student 61 is on topic T_23 is shown content C_23_3 feedback recd is 0. Expected Pay-off of this arm is 0.47, compared to expected pay-off of next round is 0.49. Actual decision was 0 with confidence -5.843074342441227. Decision of skip classifi

Student 40 is on topic T_22 is shown content C_22_8 feedback recd is 1. Expected Pay-off of this arm is 0.53, compared to expected pay-off of next round is 0.58. Actual decision was 0 with confidence -9.175781323313988. Decision of skip classifier is 0
Student 79 is on topic T_25 is shown content C_25_14 feedback recd is 0. Expected Pay-off of this arm is 0.57, compared to expected pay-off of next round is 0. Actual decision was 0 with confidence -7.048984444065547. Decision of skip classifier is 0
Student 79 is on topic T_25 is shown content C_25_4 feedback recd is 1. Expected Pay-off of this arm is 0.53, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 2.3844326815097903. Decision of skip classifier is 0
Student 68 is on topic T_25 is shown content C_25_1 feedback recd is 0. Expected Pay-off of this arm is 0.48, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 1.8931717576645752. Decision of skip classifier i

Student 51 is on topic T_25 is shown content C_25_2 feedback recd is 0. Expected Pay-off of this arm is 0.5, compared to expected pay-off of next round is 0. Actual decision was 0 with confidence -5.4182013458137686. Decision of skip classifier is 0
Student 51 is on topic T_25 is shown content C_25_1 feedback recd is 0. Expected Pay-off of this arm is 0.49, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 1.127874012856111. Decision of skip classifier is 0
Student 51 is on topic T_25 is shown content C_25_8 feedback recd is 0. Expected Pay-off of this arm is 0.47, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 1.2611341632525974. Decision of skip classifier is 0
Student 51 is on topic T_25 is shown content C_25_14 feedback recd is 1. Expected Pay-off of this arm is 0.46, compared to expected pay-off of next round is 0. Actual decision was 1 with confidence 1.3286503515956234. Decision of skip classifier is 0


In [17]:
import pandas as pd 

In [18]:
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('logs_oracle_small')
#df[df['confidence_score'] > 25]
df[df['student_number'] == 8]


FileNotFoundError: File b'logs_oracle_small' does not exist